In [1]:
from transformers import  utils, Trainer, TrainingArguments, ElectraTokenizer, ElectraForSequenceClassification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
import torch
from accelerate import Accelerator
import os
from performance import PerformanceSaver
from bertviz import model_view
utils.logging.set_verbosity_error()

In [2]:
LOAD_SAVED_MODEL = False
AUGMENT_WITH_NEUTRAL = True
saved_model_path = "models/binary/electra_classifier"
model_name = "howey/electra-base-mnli"
data_dir = "data/argumentation"

train_df = pd.read_csv(os.path.join(data_dir, 'train_iam.tsv'), sep='\t')
dev_df = pd.read_csv(os.path.join(data_dir, 'dev_iam.txt'), sep='\t')
test_df = pd.read_csv(os.path.join(data_dir, 'test_iam.txt'), sep='\t')
all_claims = pd.read_csv(os.path.join(data_dir, 'claims.txt'), sep='\t')
np.random.seed(42)

if AUGMENT_WITH_NEUTRAL:
    neutral_claims = all_claims[all_claims.type=='O'] 
    lower_bound = 0
    
    min_train_label = min(train_df['label'].value_counts())
    train_sample = neutral_claims.iloc[:min_train_label]
    train_df = pd.concat([train_df, train_sample]).sample(frac=1)
    lower_bound = min_train_label
    
    min_dev_label = min(dev_df['label'].value_counts())
    dev_sample = neutral_claims.iloc[lower_bound: lower_bound + min_dev_label]    
    dev_df = pd.concat([dev_df, dev_sample]).sample(frac=1)
    lower_bound = lower_bound + min_dev_label
    
    min_test_label = min(dev_df['label'].value_counts())
    test_sample = neutral_claims.iloc[lower_bound: lower_bound + min_test_label]    
    test_df = pd.concat([test_df, test_sample]).sample(frac=1)
    
    
label_encoder = LabelEncoder()
label_encoder.fit(train_df['label'])
train_df['label'] = label_encoder.transform(train_df['label'])
dev_df['label'] = label_encoder.transform(dev_df['label'])
test_df['label'] = label_encoder.transform(test_df['label'])

dataset = DatasetDict({
    'train': Dataset.from_pandas(train_df),
    'validation': Dataset.from_pandas(dev_df),
    'test': Dataset.from_pandas(test_df)
})

In [3]:
label_encoder.classes_

array([-1,  0,  1])

In [4]:
num_labels=len(label_encoder.classes_)
ignore_mismatched_sizes=True
classifier_dropout=0.1
output_attentions=False

if LOAD_SAVED_MODEL:
    tokenizer = ElectraTokenizer.from_pretrained(saved_model_path)
    model = ElectraForSequenceClassification.from_pretrained(saved_model_path, num_labels=num_labels, ignore_mismatched_sizes=ignore_mismatched_sizes, classifier_dropout=classifier_dropout, output_attentions=output_attentions)
else:
    tokenizer = ElectraTokenizer.from_pretrained(model_name)
    model = ElectraForSequenceClassification.from_pretrained(model_name, num_labels=num_labels, ignore_mismatched_sizes=ignore_mismatched_sizes, classifier_dropout=classifier_dropout, output_attentions=output_attentions)
    
if torch.cuda.is_available():
        model = model.to(device='cuda')

In [5]:
def process(batch):
    inputs = tokenizer(batch["argument"], truncation=True, padding="max_length")
    return {
            "input_ids": inputs["input_ids"],
            "attention_mask": inputs["attention_mask"],
            "labels": batch["label"],
        }
    
tokenized_dataset = dataset.map(process, batched=True, remove_columns=['type', 'id'])

Map:   0%|          | 0/5644 [00:00<?, ? examples/s]

Map:   0%|          | 0/725 [00:00<?, ? examples/s]

Map:   0%|          | 0/760 [00:00<?, ? examples/s]

In [6]:
training_args = TrainingArguments(
            output_dir='./results',
            do_eval=True,
            do_train=True,
            num_train_epochs=6,
            save_total_limit=2,
            load_best_model_at_end=True,
            learning_rate=1e-03,
            per_device_train_batch_size=8,
            per_device_eval_batch_size=8,
            save_strategy="steps",
            logging_strategy="steps",
            evaluation_strategy="steps",
            logging_steps=60,
            eval_steps=60,
            save_steps=60,
        )

def compute_metrics(pred):
            print(pred)
            print("======================================")
            labels = pred.label_ids
            preds = pred.predictions.argmax(-1)
            precision, recall, f1, _ = precision_recall_fscore_support(
                labels, preds, average="weighted"
            )
            acc = accuracy_score(labels, preds)
            return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [7]:
trainer = Trainer(
            model=model,
            args=training_args,
            compute_metrics=compute_metrics,
            train_dataset=tokenized_dataset["train"],
            eval_dataset=tokenized_dataset["validation"],
            tokenizer=tokenizer,
        )

In [8]:
from tqdm.notebook import tqdm
if not LOAD_SAVED_MODEL:
    trainer.train()
    trainer.save_model("models/binary/electra_classifier")
    predictions = trainer.predict(tokenized_dataset["test"])
    print(predictions.metrics)

/root/lucaM/LogicalFallacyRepo/logical-fallacy-LLMs/logicalfallacies/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'loss': 1.1825, 'learning_rate': 0.0009716713881019831, 'epoch': 0.17}
{'eval_loss': 1.1063247919082642, 'eval_accuracy': 0.35724137931034483, 'eval_f1': 0.18805999439304738, 'eval_precision': 0.12762140309155767, 'eval_recall': 0.35724137931034483, 'eval_runtime': 14.0959, 'eval_samples_per_second': 51.434, 'eval_steps_per_second': 3.263, 'epoch': 0.17}


/root/lucaM/LogicalFallacyRepo/logical-fallacy-LLMs/logicalfallacies/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/lucaM/LogicalFallacyRepo/logical-fallacy-LLMs/logicalfallacies/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'loss': 1.1028, 'learning_rate': 0.000943342776203966, 'epoch': 0.34}
{'eval_loss': 1.1005462408065796, 'eval_accuracy': 0.35724137931034483, 'eval_f1': 0.18805999439304738, 'eval_precision': 0.12762140309155767, 'eval_recall': 0.35724137931034483, 'eval_runtime': 14.0942, 'eval_samples_per_second': 51.44, 'eval_steps_per_second': 3.264, 'epoch': 0.34}


/root/lucaM/LogicalFallacyRepo/logical-fallacy-LLMs/logicalfallacies/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/lucaM/LogicalFallacyRepo/logical-fallacy-LLMs/logicalfallacies/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


...

/root/lucaM/LogicalFallacyRepo/logical-fallacy-LLMs/logicalfallacies/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/lucaM/LogicalFallacyRepo/logical-fallacy-LLMs/logicalfallacies/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'loss': 1.0845, 'learning_rate': 0.0006883852691218131, 'epoch': 1.87}
{'eval_loss': 1.1115469932556152, 'eval_accuracy': 0.35724137931034483, 'eval_f1': 0.18805999439304738, 'eval_precision': 0.12762140309155767, 'eval_recall': 0.35724137931034483, 'eval_runtime': 14.1217, 'eval_samples_per_second': 51.339, 'eval_steps_per_second': 3.257, 'epoch': 1.87}


/root/lucaM/LogicalFallacyRepo/logical-fallacy-LLMs/logicalfallacies/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/lucaM/LogicalFallacyRepo/logical-fallacy-LLMs/logicalfallacies/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'loss': 1.1032, 'learning_rate': 0.000660056657223796, 'epoch': 2.04}
{'eval_loss': 1.0975288152694702, 'eval_accuracy': 0.35724137931034483, 'eval_f1': 0.18805999439304738, 'eval_precision': 0.12762140309155767, 'eval_recall': 0.35724137931034483, 'eval_runtime': 14.0906, 'eval_samples_per_second': 51.453, 'eval_steps_per_second': 3.265, 'epoch': 2.04}


/root/lucaM/LogicalFallacyRepo/logical-fallacy-LLMs/logicalfallacies/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/lucaM/LogicalFallacyRepo/logical-fallacy-LLMs/logicalfallacies/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
